# Meep FDTD

[Meep](https://meep.readthedocs.io/en/latest/) is a free and open source finite-difference time-domain (FDTD) software package for electromagnetic simulations spanning a broad range of applications.

You can install Meep and the mode solver MPB (which Meep uses to compute S-parameters and launch mode sources) using one of two methods:

1. Mamba (faster)

```
mamba install pymeep=*=mpi_mpich_* -y
```

2. Conda

```
conda install -c conda-forge pymeep=*=mpi_mpich_* -y
```

To update the installed package, you would do:

```
mamba update pymeep=*=mpi_mpich_* -y
```

The Mamba/Conda packages are available for Linux, macOS, or Windows [WSL](https://docs.microsoft.com/en-us/windows/wsl/). For more details on installing Meep, see the [user manual](https://meep.readthedocs.io/en/latest/Installation/#conda-packages).


The gdsfactory `gmeep` plugin computes the transmission spectrum for planar photonic components.

One advantage of using the `gmeep` plugin is that you only need to define your component geometry *once* using gdsfactory. The geometry is automatically imported into Meep. There is no need to define the geometry separately for Meep.

For extracting S-parameters, `gmeep` automatically swaps the source between ports to compute the full S-matrix. This process involves:

- adding monitors to each port of the device
- extending the ports into the PML absorbing boundary layers
- running the simulation and computing elements of the S-matrix in post-processing using the correct ratio of S-parameters. The port monitors are used to compute the discrete-time Fourier-transformed (DTFT) fields which are then decomposed into complex mode coefficients known as S-parameters. The S-parameters can be computed over a range of frequencies.

The resolution is in units of pixels/μm. As a general rule, you should run with at least `resolution=30` for 1/30 μm/pixel (33 nm/pixel)

Note that most of the examples use `resolution=20` in order to run fast.

Here are some examples of how to extract S-parameters in Meep in planar devices.

For Sparameters we follow the syntax `o1@0,o2@0` where `o1` is the input port `@0` mode 0 (usually fundamental TE mode) and `o2@0` refers to output port `o2` mode 0. 


```bash

         top view
              ________________________________
             |                               |
             | xmargin_left                  | port_extension
             |<--------->       port_margin ||<-->
          o2_|___________          _________||_o3
             |           \        /          |
             |            \      /           |
             |             ======            |
             |            /      \           |
          o1_|___________/        \__________|_o4
             |   |                 <-------->|
             |   |ymargin_bot   xmargin_right|
             |   |                           |
             |___|___________________________|

        side view
              ________________________________
             |                     |         |
             |                     |         |
             |                   zmargin_top |
             |xmargin_left         |         |
             |<---> _____         _|___      |
             |     |     |       |     |     |
             |     |     |       |     |     |
             |     |_____|       |_____|     |
             |       |                       |
             |       |                       |
             |       |zmargin_bot            |
             |       |                       |
             |_______|_______________________|



```

## Serial Simulation (single core)

Running Meep using a single CPU core can be slow as the single core needs to update all the simulation grid points sequentially at each time step.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import gdsfactory as gf
import gdsfactory.simulation.gmeep as gm
import gdsfactory.simulation as sim

gf.config.set_plot_options(show_subports=False, show_ports=False)

In [ ]:
c = gf.components.straight(length=2)
c

`run=False` only plots the simulations for you to review that is set up **correctly**

In [ ]:
df = gm.write_sparameters_meep(c, run=False, ymargin_top=3, ymargin_bot=3)

In [ ]:
help(gm.write_sparameters_meep)

As you've noticed we added `ymargin_top` and `ymargin_bot` to ensure we have enough distance to the PML

You can also do this directly with another version of the function that adds `ymargin_top` and `ymargin_bot`

In [ ]:
c = gf.components.straight(length=2)
df = gm.write_sparameters_meep(c, run=False)

Because components with `left-right` ports are very common `write_sparameters_meep` `y_margin = 3um `

In [ ]:
c = gf.components.taper(length=2.0, width1=0.5, width2=1)
c

In [ ]:
df = gm.write_sparameters_meep(c, run=False)

## 2.5D Simulation

For faster simulations you can do an effective mode approximation, to compute the mode of the slab and run a 2D simulation to speed your [simulations](https://www.lumerical.com/learn/whitepapers/lumericals-2-5d-fdtd-propagation-method/)

In [ ]:
ncore = sim.get_effective_indices(
    ncore=3.4777,
    ncladding=1.444,
    nsubstrate=1.444,
    thickness=0.22,
    wavelength=1.55,
    polarization="te",
)[0]

ncore

In [ ]:
df = gm.write_sparameters_meep(
    c, resolution=20, is_3d=False, material_name_to_meep=dict(si=ncore)
)

In [ ]:
gf.simulation.plot.plot_sparameters(df)

In [ ]:
gf.simulation.plot.plot_sparameters(df, keys=("o2@0,o1@0",))

For a small taper length, the matrix element S$_{21}$ (transmission in Port 2 given a source in Port 1) is around 0 dB which is equivalent to ~100% transmission.

## Port Symmetries

You can skip redundant simulations in reciprocal devices.
If the device looks the same going from in -> out as out -> in, we just need to run one simulation.

In [ ]:
c = gf.components.bend_euler(radius=3)
c

In [ ]:
sp = gm.write_sparameters_meep_1x1_bend90(c, run=False)

In [ ]:
sp = gm.write_sparameters_meep_1x1_bend90(c, run=True)

In [ ]:
list(sp.keys())

In [ ]:
gf.simulation.plot.plot_sparameters(sp)

In [ ]:
gf.simulation.plot.plot_sparameters(df, keys=("o2@0,o1@0",), logscale=False)

In [ ]:
gf.simulation.plot.plot_sparameters(df, keys=("o2@0,o1@0",))

In [ ]:
c = gf.components.crossing()
c

Here are the port symmetries for a crossing

```python
port_symmetries_crossing = {
    "o1": {
        "o1@0,o1@0": ["o2@0,o2@0", "o3@0,o3@0", "o4@0,o4@0"],
        "o2@0,o1@0": ["o1@0,o2@0", "o3@0,o4@0", "o4@0,o3@0"],
        "o3@0,o1@0": ["o1@0,o3@0", "o2@0,o4@0", "o4@0,o2@0"],
        "o4@0,o1@0": ["o1@0,o4@0", "o2@0,o3@0", "o3@0,o2@0"],
    }
}
```

In [ ]:
sp = gm.write_sparameters_meep(
    c,
    resolution=20,
    ymargin=0,
    port_symmetries=gm.port_symmetries.port_symmetries_crossing,
    run=False,
)

In [ ]:
sp = gm.write_sparameters_meep(
    c,
    resolution=20,
    ymargin=0,
    port_symmetries=gm.port_symmetries.port_symmetries_crossing,
    run=True,
)

In [ ]:
gm.plot.plot_sparameters(sp)

In [ ]:
gm.plot.plot_sparameters(sp, keys=("o3@0,o1@0",))

As you can see this crossing looks beautiful but is quite **lossy** (9 dB @ 1550 nm)

## Parallel Simulation (multicore/MPI)

Meep supports [distributed-memory parallelization via MPI](https://meep.readthedocs.io/en/latest/Parallel_Meep/) which can be used to provide a significant speedup compared to serial calculations.

By default MPI just runs the same copy of the Python script everywhere, with the C++ under MEEP actually being parallelized.

`divide_parallel_processes` allows us to logically split this one calculation into (in this case "cores") subdivisions.
The only difference in the scripts is that a different integer n is returned depending on the subdivision it is running in.

So we use that n to select different sources, and each subdivision calculates its own Sparams independently.
Afterwards, we collect all the results in one of the subdivisions (if rank == 0)

As a demonstration, lets try to reproduce the results of the directional coupler results from the [Meep manual](https://meep.readthedocs.io/en/latest/Python_Tutorials/GDSII_Import/) which indicates that to obtain a 3 dB (50%/50%) splitter you need a separation distance of 130 nm over a coupler length of 8 μm.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

import gdsfactory as gf
import gdsfactory.simulation as sim
import gdsfactory.simulation.gmeep as gm

In [ ]:
help(gf.components.coupler)

In [ ]:
c = gf.components.coupler(length=8, gap=0.13)
c

In [ ]:
gm.write_sparameters_meep(component=c, run=False)

In [ ]:
filepath = gm.write_sparameters_meep_mpi(
    component=c,
    cores=4,
    resolution=30,
)

In [ ]:
sp = np.load(filepath)

In [ ]:
gf.simulation.plot.plot_sparameters(sp)

In [ ]:
gf.simulation.plot.plot_sparameters(sp, keys=["o1@0,o3@0", "o1@0,o4@0"])

## Batch Simulations

You can also run a batch of multicore simulations.

Given a list of write_sparameters_meep keyword arguments `jobs` launches them in different cores using MPI where each simulation runs with "cores_per_run" cores.

If there are more simulations than cores each batch runs sequentially.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import gdsfactory as gf

import gdsfactory.simulation as sim
import gdsfactory.simulation.gmeep as gm

In [ ]:
help(gm.write_sparameters_meep_batch)

In [ ]:
c = gf.components.straight(length=3.1)

In [ ]:
gm.write_sparameters_meep(c, ymargin=3, run=False)

In [ ]:
c1_dict = {"component": c, "ymargin": 3}
jobs = [
    c1_dict,
]

filepaths = gm.write_sparameters_meep_batch_1x1(
    jobs=jobs,
    cores_per_run=4,
    total_cores=8,
    lazy_parallelism=True,
)

In [ ]:
sp = np.load(filepaths[0])
gf.simulation.plot.plot_sparameters(sp)

In [ ]:
c = gf.components.coupler_ring()
c

In [ ]:
p = 2.5
gm.write_sparameters_meep(c, ymargin=0, ymargin_bot=p, xmargin=p, run=False)

In [ ]:
c1_dict = dict(
    component=c,
    ymargin=0,
    ymargin_bot=p,
    xmargin=p,
)
jobs = [c1_dict]

filepaths = gm.write_sparameters_meep_batch(
    jobs=jobs,
    cores_per_run=4,
    total_cores=8,
    delete_temp_files=False,
    lazy_parallelism=True,
)

In [ ]:
sp = np.load(filepaths[0])

In [ ]:
gm.plot.plot_sparameters(sp)

In [ ]:
gm.plot.plot_sparameters(sp, keys=['o3@0,o1@0', 'o4@0,o1@0'])

In [ ]:
gm.plot.plot_sparameters(sp, keys=["s31"], with_sparameter_keys=True)

In [ ]:
gm.plot.plot_sparameters(sp, keys=["s41"], with_sparameter_keys=True)

## Visualizing the 3D Geometry

In [ ]:
from gdsfactory.simulation.add_simulation_markers import add_simulation_markers

c = gf.components.mmi1x2()
c = add_simulation_markers(c)
c

In [ ]:
scene = c.to_3d()
scene.show()